In [1]:
# packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

### Kenneth French 'Market Equity'-data. 
Now: The portfolios, which are constructed monthly, are the intersections of 5 portfolios formed on size (market equity, ME) and 5 portfolios formed on prior (2-12) return. The monthly size breakpoints are the NYSE market equity quintiles. The monthly prior (2-12) return breakpoints are NYSE quintiles.

The portfolios constructed each month include NYSE, AMEX, and NASDAQ stocks with prior return data. To be included in a portfolio for month t in the monthly returns (formed at the end of month t-1), a stock must have a price for the end of month t-13 and a good return for t-2. In addition, any missing returns from t-12 to t-3 must be -99.0, CRSP's code for a missing price. Each included stock also must have ME for the end of month t-1.

### Average Value Weighted Returns -- Monthly:

In [2]:
pf_25_vwr = pd.read_csv("Data/25_Portfolios_ME_Prior_12_2.csv", skiprows=11, nrows = 1171)
pf_25_vwr = pf_25_vwr.rename(columns = {"Unnamed: 0" : "Date"})
pf_25_vwr["Date"] = pd.to_datetime(pf_25_vwr["Date"], format="%Y%m") + pd.offsets.MonthEnd()
pf_25_vwr = pf_25_vwr[(pf_25_vwr["Date"]<=pd.Timestamp('2023-12-31')) & (pf_25_vwr["Date"]>=pd.Timestamp('1927-01-01'))] 
pf_25_vwr = pf_25_vwr.set_index("Date").sort_index(ascending=False)

In [3]:
missing_values_vwr = [-99.99, -999]
missing_count_vwr = pf_25_vwr.isin(missing_values_vwr).sum()
missing_count_vwr = missing_count_vwr[missing_count_vwr > 0]
#na_count_vwr = pf_25_vwr.isna().sum()
#empty_count_vwr = (pf_25_vwr == '').sum()

#print(missing_count_vwr, na_count_vwr, empty_count_vwr)
print(missing_count_vwr)

BIG LoPRIOR    4
dtype: int64


Handling missing values:

In [5]:
pf_25_vwr.replace(missing_values_vwr, np.nan, inplace=True)
column_means = pf_25_vwr.mean()
pf_25_vwr.fillna(column_means, inplace=True)

### Number of Firms in Portfolios - Monthly:

In [6]:
pf_25_nf = pd.read_csv("Data/25_Portfolios_ME_Prior_12_2.csv", skiprows=2563, nrows = 1171)
pf_25_nf = pf_25_nf.rename(columns = {"Unnamed: 0" : "Date"})
pf_25_nf["Date"] = pd.to_datetime(pf_25_nf["Date"], format="%Y%m") + pd.offsets.MonthEnd()
pf_25_nf = pf_25_nf[(pf_25_nf["Date"]<=pd.Timestamp('2023-12-31')) & (pf_25_nf["Date"]>=pd.Timestamp('1927-01-01'))] 
pf_25_nf = pf_25_nf.set_index("Date").sort_index(ascending=False)

In [7]:
missing_values_nf = [-99.99, -999]
missing_count_nf = pf_25_nf.isin(missing_values_nf).sum()
missing_count_nf = missing_count_nf[missing_count_nf > 0]

#na_count_nf = pf_25_nf.isna().sum()
#empty_count_nf = (pf_25_nf == '').sum()

#print(missing_count_nf, na_count_nf, empty_count_nf)
print(missing_count_nf)

Series([], dtype: int64)


### Average Firm Size - Monthly:

In [8]:
pf_25_afs = pd.read_csv("Data/25_Portfolios_ME_Prior_12_2.csv", skiprows=3738, nrows = 1171)
pf_25_afs = pf_25_afs.rename(columns = {"Unnamed: 0" : "Date"})
pf_25_afs["Date"] = pd.to_datetime(pf_25_afs["Date"], format="%Y%m") + pd.offsets.MonthEnd()
pf_25_afs = pf_25_afs[(pf_25_afs["Date"]<=pd.Timestamp('2023-12-31')) & (pf_25_afs["Date"]>=pd.Timestamp('1927-01-01'))] 
pf_25_afs = pf_25_afs.set_index("Date").sort_index(ascending=False)

In [9]:
missing_values_afs = [-99.99, -999]
missing_count_afs = pf_25_afs.isin(missing_values_afs).sum()
missing_count_afs = missing_count_afs[missing_count_afs > 0]

#na_count_afs = pf_25_afs.isna().sum()
#empty_count_afs = (pf_25_afs == '').sum()

print(missing_count_afs)
#print(missing_count_afs, missing_count_afs, empty_count_afs)

BIG LoPRIOR    4
dtype: int64


Handling missing values:

In [10]:
pf_25_afs.replace(missing_values_afs, np.nan, inplace=True)
column_means = pf_25_afs.mean()
pf_25_afs.fillna(column_means, inplace=True)

Callculate entire stock market return:

In [11]:
market_cap = pf_25_nf.multiply(pf_25_afs)
total_market_cap = market_cap.sum(axis=1)
weights = market_cap.divide(total_market_cap, axis=0)
weighted_returns = pf_25_vwr.multiply(weights)
total_market_return = weighted_returns.sum(axis=1)
pf_25_returns = pf_25_vwr.copy()
pf_25_returns['Market Return'] = weighted_returns.sum(axis=1).round(2)

Gem rensede filer:

In [15]:
pf_25_returns.sort_index(ascending=True, inplace=True)
pf_25_afs.sort_index(ascending=True, inplace=True)
pf_25_nf.sort_index(ascending=True, inplace=True)

In [16]:
pf_25_returns.to_csv("Data_clean/25_Portfolios_ME_Prior_12_2_returns.csv")
pf_25_afs.to_csv("Data_clean/25_Portfolios_ME_Prior_12_2_afs.csv")
pf_25_nf.to_csv("Data_clean/25_Portfolios_ME_Prior_12_2_nf.csv")